In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
from scipy import stats
from scipy.stats import norm, skew

In [10]:
import seaborn as sb
plt.style.use('seaborn')
sb.set(font_scale=2.5)

In [11]:
#EDA
#possible Missing values for columns
miss_val=['NaN']

In [12]:
#Loading Data
aisles=pd.read_csv('../instacart-market-basket-analysis-files/aisles.csv', na_values=miss_val)
departments=pd.read_csv('../instacart-market-basket-analysis-files/departments.csv', na_values=miss_val)
products=pd.read_csv('../instacart-market-basket-analysis-files/products.csv', na_values=miss_val)
order_products_prior=pd.read_csv('../instacart-market-basket-analysis-files/order_products__prior.csv', na_values=miss_val)
order_products_train=pd.read_csv('../instacart-market-basket-analysis-files/order_products__train.csv', na_values=miss_val)
orders=pd.read_csv('../instacart-market-basket-analysis-files/orders.csv', na_values=miss_val)

print('train:', order_products_prior.shape, 'test:', order_products_train.shape)

train: (32434489, 4) test: (1384617, 4)


In [13]:
aisles.count

<bound method DataFrame.count of      aisle_id                       aisle
0           1       prepared soups salads
1           2           specialty cheeses
2           3         energy granola bars
3           4               instant foods
4           5  marinades meat preparation
..        ...                         ...
129       130    hot cereal pancake mixes
130       131                   dry pasta
131       132                      beauty
132       133  muscles joints pain relief
133       134  specialty wines champagnes

[134 rows x 2 columns]>

In [14]:
departments.isnull().values.any()
departments.head(15)

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [15]:
products.isnull().values.any()
products.head(5)

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [16]:
order_products_prior.isnull().values.any()
order_products_prior.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 4 columns):
order_id             int64
product_id           int64
add_to_cart_order    int64
reordered            int64
dtypes: int64(4)
memory usage: 989.8 MB


In [17]:
order_products_train.isnull().values.any()
order_products_train.head(5)

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [18]:
orders.head(10)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0
6,550135,1,prior,7,1,9,20.0
7,3108588,1,prior,8,1,14,14.0
8,2295261,1,prior,9,1,16,0.0
9,2550362,1,prior,10,4,8,30.0


In [19]:
orders['days_since_prior_order'].isnull().any()

True

In [20]:
# Data Preprocessing

In [21]:
orders.fillna(-1, inplace=True) # Fill missing values with -1
orders.head(4)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,-1.0
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0


In [22]:
print(orders['days_since_prior_order'].describe())

count    3.421083e+06
mean     1.038460e+01
std      9.379136e+00
min     -1.000000e+00
25%      4.000000e+00
50%      7.000000e+00
75%      1.500000e+01
max      3.000000e+01
Name: days_since_prior_order, dtype: float64


In [23]:
# Combining dataframes orders_prior and orders_train
order_products_total=pd.concat([order_products_prior, order_products_train], axis=0)
order_products_total.shape
order_products_total.head(5)

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [24]:
order_products_usr=pd.merge(orders, order_products_total, on='order_id')
order_products_usr.head(10)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,-1.0,196,1,0
1,2539329,1,prior,1,2,8,-1.0,14084,2,0
2,2539329,1,prior,1,2,8,-1.0,12427,3,0
3,2539329,1,prior,1,2,8,-1.0,26088,4,0
4,2539329,1,prior,1,2,8,-1.0,26405,5,0
5,2398795,1,prior,2,3,7,15.0,196,1,1
6,2398795,1,prior,2,3,7,15.0,10258,2,0
7,2398795,1,prior,2,3,7,15.0,12427,3,1
8,2398795,1,prior,2,3,7,15.0,13176,4,0
9,2398795,1,prior,2,3,7,15.0,26088,5,1


In [25]:
order_products_usr.tail(1)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
33819105,272231,206209,train,14,6,14,30.0,39216,8,1


In [26]:
order_products_usr['order_number'].nlargest(10, keep='last').values

array([100, 100, 100, 100, 100, 100, 100, 100, 100, 100])

In [27]:
order_products_usr['order_number'].max()

100

In [28]:
# Data Visualization

In [29]:
#Product Popularity by ALL users
most_freq_prod=order_products_usr.groupby('product_id').size().sort_values(ascending=False).reset_index()
#most_freq_prod.rename(columns={"product_id": "product_ID", "0": "product_count"}, inplace=True)
print(most_freq_prod)

       product_id       0
0           24852  491291
1           13176  394930
2           21137  275577
3           21903  251705
4           47209  220877
...           ...     ...
49680        7626       1
49681       14485       1
49682       43098       1
49683       11254       1
49684       36233       1

[49685 rows x 2 columns]


In [30]:
# Total count of each product for every user
freq_user_product=order_products_usr.groupby('user_id')['product_id'].value_counts()
    #freq_user_product[i]=order_products_usr[order_products_usr['user_id']==i]['product_id'].value_counts()
    #most_freq_product_user[i]=freq_user_product[i].index[0]
    #print("Most frequent product for User {} is {}\n\n".format(i,most_freq_product_user[i]))
print(freq_user_product)

user_id  product_id
1        196           11
         10258         10
         12427         10
         25133          9
         13032          4
                       ..
206209   41665          1
         42606          1
         44325          1
         48370          1
         48697          1
Name: product_id, Length: 13863746, dtype: int64


In [31]:
# Total number of unique orders for every user
total_orders_user=order_products_usr.groupby('user_id')['order_id'].nunique().reset_index().rename(columns={'order_id':'total_unique_orders'})
print(total_orders_user)

        user_id  total_unique_orders
0             1                   11
1             2                   15
2             3                   12
3             4                    5
4             5                    5
...         ...                  ...
206204   206205                    4
206205   206206                   67
206206   206207                   16
206207   206208                   49
206208   206209                   14

[206209 rows x 2 columns]


In [32]:
# Total number of orders for every user
total_orders_user2=order_products_usr.groupby('user_id')['order_id'].count().reset_index().rename(columns={'order_id':'count_orders'})
#print(total_orders_user2)

# Product details of every order for every user
total_orders_user3=order_products_usr.groupby('user_id')['order_id'].value_counts() 
print(total_orders_user3)

user_id  order_id
1        1187899     11
         2550362      9
         431534       8
         2295261      6
         2398795      6
                     ..
206209   272231       8
         1854736      8
         2307371      3
         2558525      3
         3186442      2
Name: order_id, Length: 3346083, dtype: int64


In [33]:
torders=0
for key, value in total_orders_user3.items():
    if (key[0]==1):
        torders+=value

print(torders)

70


In [34]:
# Unique Product Count for every user
unique_product_user=order_products_usr.groupby('user_id')['product_id'].nunique().reset_index().rename(columns={'product_id':'num_unique_products'})
print(unique_product_user)    

        user_id  num_unique_products
0             1                   19
1             2                  121
2             3                   33
3             4                   17
4             5                   28
...         ...                  ...
206204   206205                   37
206205   206206                  150
206206   206207                   92
206207   206208                  198
206208   206209                   72

[206209 rows x 2 columns]


In [35]:
#new_product_ratio for every user = unique products/total orders

nd=pd.merge(unique_product_user, total_orders_user2, on="user_id")
nd['new_product_ratio'] = nd.apply(lambda row: row.num_unique_products/row.count_orders*100, axis=1)
print(nd)

        user_id  num_unique_products  count_orders  new_product_ratio
0             1                   19            70          27.142857
1             2                  121           226          53.539823
2             3                   33            88          37.500000
3             4                   17            18          94.444444
4             5                   28            46          60.869565
...         ...                  ...           ...                ...
206204   206205                   37            51          72.549020
206205   206206                  150           285          52.631579
206206   206207                   92           223          41.255605
206207   206208                  198           677          29.246677
206208   206209                   72           137          52.554745

[206209 rows x 4 columns]


In [36]:
#Product ordering ratio for every user = total number of orders containing product/ total number of orders
#How many times a user has reordered a product?

order_contains_product=order_products_usr.groupby(['user_id', 'product_id'])['order_id'].agg(['count']).reset_index().rename(columns={'count': "orders_with_product"})
total_user_order=order_products_usr.groupby('user_id')['order_id'].agg(['count']).reset_index().rename(columns={'count': "total_orders"})
nro=pd.merge(order_contains_product, total_user_order, on="user_id")
nro['product_order_ratio']=nro.apply(lambda row: row.orders_with_product/row.total_orders*100, axis=1)
print(nro)

          user_id  product_id  orders_with_product  total_orders  \
0               1         196                   11            70   
1               1       10258                   10            70   
2               1       10326                    1            70   
3               1       12427                   10            70   
4               1       13032                    4            70   
...           ...         ...                  ...           ...   
13863741   206209       43961                    3           137   
13863742   206209       44325                    1           137   
13863743   206209       48370                    1           137   
13863744   206209       48697                    1           137   
13863745   206209       48742                    2           137   

          product_order_ratio  
0                   15.714286  
1                   14.285714  
2                    1.428571  
3                   14.285714  
4                    5.

In [37]:
print(total_user_order.iloc[0])

user_id          1
total_orders    70
Name: 0, dtype: int64


In [38]:
# Frequency of products ordered by every user in each order
#user_order_products22=order_products_usr.groupby(['user_id', 'order_id'])['product_id'].count() # returns a Series
user_order_products=order_products_usr.groupby(['user_id', 'order_id'])['product_id'].agg(['count',  'mean']) # returns a Dataframe
for a in user_order_products.itertuples():
    print(a[0], a[1], a[2])
    break

(1, 431534) 8 16303.125


In [39]:
# User Shopping times by popularity for every user
user_time=order_products_usr.groupby('user_id')['order_hour_of_day'].value_counts()
print(user_time)

user_id  order_hour_of_day
1        8                    25
         7                    15
         15                    8
         14                    6
         16                    6
                              ..
206209   13                   10
         18                    9
         14                    8
         15                    6
         16                    2
Name: order_hour_of_day, Length: 1540304, dtype: int64


In [40]:
# User Shopping times by popularity for all users
most_popular_time=order_products_usr.groupby('order_hour_of_day')['user_id'].count()
print(most_popular_time)

order_hour_of_day
0      228031
1      121412
2       72660
3       53759
4       55714
5       91909
6      302642
7      928239
8     1787359
9     2550569
10    2874905
11    2852701
12    2732599
13    2778054
14    2810918
15    2780731
16    2647695
17    2186409
18    1714445
19    1317576
20    1017958
21     831183
22     662053
23     419585
Name: user_id, dtype: int64


In [41]:
# User Shopping Days by popularity for every user
user_day=order_products_usr.groupby('user_id')['order_dow'].value_counts()
print(user_day)

user_id  order_dow
1        4            33
         1            17
         3            11
         2             9
2        1            95
                      ..
206209   5            24
         4            14
         2            10
         6             8
         0             2
Name: order_dow, Length: 997461, dtype: int64


In [42]:
# User Shopping Day Trends for all users
most_popular_day=order_products_usr.groupby('order_dow')['user_id'].count()
print(most_popular_day)

order_dow
0    6533692
1    5871834
2    4378360
3    3998498
4    3942696
5    4386443
6    4707583
Name: user_id, dtype: int64


In [43]:
order_products_usr.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,-1.0,196,1,0
1,2539329,1,prior,1,2,8,-1.0,14084,2,0
2,2539329,1,prior,1,2,8,-1.0,12427,3,0
3,2539329,1,prior,1,2,8,-1.0,26088,4,0
4,2539329,1,prior,1,2,8,-1.0,26405,5,0


In [49]:
#Most ordered product in every department
products_departments=pd.merge(products, departments, on='department_id')
total_orders_departments=pd.merge(order_products_usr, products_departments,on="product_id")
department_popular_products=total_orders_departments.groupby('department_id')['product_id'].count()
department_popular_products.head(12)

department_id
1     2336858
2       38086
3     1225181
4     9888378
5      159294
6      281155
7     2804175
8      102221
9      905340
10      35932
11     468693
12     739238
Name: product_id, dtype: int64

In [45]:
#Did any user reorder their previous order

In [46]:
#Times a customer orders from the same department. Wrt to customer

In [47]:
#Days a customer orders the same products

In [48]:
#Times a product is ordered from the same department. Wrt product